In [1]:
#!pip install spotipy
#!pip install bs4

In [2]:
import pandas as pd
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
from bs4 import BeautifulSoup
import requests
import json
from pandas.io.json import json_normalize

## Spotify api credentials

In [3]:
#Authentication - without user
client_credentials_manager = SpotifyClientCredentials(client_id="3a2f65eb20164b24b59b7fdd6876bb51", client_secret="9f6d3a070cdf4eb8adda14a794d60469")
sp = spotipy.Spotify(client_credentials_manager = client_credentials_manager)

## Fetch data from playlist or single song

In [4]:
playlist_link = "https://open.spotify.com/playlist/2GxgTnPCRkKl6sTX1INdAw?si=74fe4ed6868b408a"
playlist_URI = playlist_link.split("/")[-1].split("?")[0]
track_uris = [x["track"]["uri"] for x in sp.playlist_tracks(playlist_URI)["items"]]

In [5]:
for track in sp.playlist_tracks(playlist_URI)["items"]:
    
    #URI
    track_uri = track["track"]["uri"]
    
    #Track name
    track_name = track["track"]["name"]
    
    #Main Artist
    artist_uri = track["track"]["artists"][0]["uri"]
    artist_info = sp.artist(artist_uri)
    
    #Name, popularity, genre
    artist_name = track["track"]["artists"][0]["name"]
    artist_pop = artist_info["popularity"]
    artist_genres = artist_info["genres"]
    
    #Album
    album = track["track"]["album"]["name"]
    
    
    
    #Popularity of the track
    track_pop = track["track"]["popularity"]
    
    trackInfo = track["track"]    
    #print(trackInfo.keys())
    
    print(track["track"])
    #Printin details
    #print(track_name,"\n", 
    #      artist_name,"\n",
    #      sp.audio_features(track_uri)[0], "\n", 
    #      track_uri
    #     )
    

{'album': {'album_type': 'single', 'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/53XhwfbYqKCa1cC15pYq2q'}, 'href': 'https://api.spotify.com/v1/artists/53XhwfbYqKCa1cC15pYq2q', 'id': '53XhwfbYqKCa1cC15pYq2q', 'name': 'Imagine Dragons', 'type': 'artist', 'uri': 'spotify:artist:53XhwfbYqKCa1cC15pYq2q'}, {'external_urls': {'spotify': 'https://open.spotify.com/artist/57nPqD7z62gDdq37US9XJR'}, 'href': 'https://api.spotify.com/v1/artists/57nPqD7z62gDdq37US9XJR', 'id': '57nPqD7z62gDdq37US9XJR', 'name': 'Arcane', 'type': 'artist', 'uri': 'spotify:artist:57nPqD7z62gDdq37US9XJR'}, {'external_urls': {'spotify': 'https://open.spotify.com/artist/47mIJdHORyRerp4os813jD'}, 'href': 'https://api.spotify.com/v1/artists/47mIJdHORyRerp4os813jD', 'id': '47mIJdHORyRerp4os813jD', 'name': 'League of Legends', 'type': 'artist', 'uri': 'spotify:artist:47mIJdHORyRerp4os813jD'}], 'available_markets': ['AD', 'AE', 'AG', 'AL', 'AM', 'AO', 'AR', 'AT', 'AU', 'AZ', 'BA', 'BB', 'BD', 'BE', '

### https://stmorse.github.io/journal/spotify-api.html

In [6]:
AUTH_URL = 'https://accounts.spotify.com/api/token'

# POST
auth_response = requests.post(AUTH_URL, {
    'grant_type': 'client_credentials',
    'client_id': "3a2f65eb20164b24b59b7fdd6876bb51",
    'client_secret': "9f6d3a070cdf4eb8adda14a794d60469",
})

# convert the response to JSON
auth_response_data = auth_response.json()

# save the access token
access_token = auth_response_data['access_token']

In [7]:
headers = {
    'Authorization': 'Bearer {token}'.format(token=access_token)
}

In [15]:
# base URL of all Spotify API endpoints
BASE_URL = 'https://api.spotify.com/v1/'

# Track ID from the URI
track_id = '5UlnuulVAVmmesw4VzqHdG'

# actual GET request with proper header
r = requests.get(BASE_URL + 'audio-features/' + track_id, headers=headers)

In [16]:
r = r.json()
r

{'danceability': 0.722,
 'energy': 0.745,
 'key': 7,
 'loudness': -4.164,
 'mode': 1,
 'speechiness': 0.256,
 'acousticness': 0.247,
 'instrumentalness': 0,
 'liveness': 0.419,
 'valence': 0.52,
 'tempo': 77.029,
 'type': 'audio_features',
 'id': '5UlnuulVAVmmesw4VzqHdG',
 'uri': 'spotify:track:5UlnuulVAVmmesw4VzqHdG',
 'track_href': 'https://api.spotify.com/v1/tracks/5UlnuulVAVmmesw4VzqHdG',
 'analysis_url': 'https://api.spotify.com/v1/audio-analysis/5UlnuulVAVmmesw4VzqHdG',
 'duration_ms': 173381,
 'time_signature': 4}

***

In [28]:
play_list = '37i9dQZEVXbMDoHDwVN2tF'

# pull all artists albums
r = requests.get(BASE_URL + 'playlists/' + artist_id, 
                 headers=headers, 
                 params={'include_groups': 'album', 'limit': 50})
d = r.json()

NameError: name 'artist_id' is not defined

***

In [30]:
def call_playlist(creator, playlist_id):
    
    #step1

    playlist_features_list = ["artist","album","track_name",  "track_id","danceability","energy","key","loudness","mode", "speechiness","instrumentalness","liveness","valence","tempo", "duration_ms","time_signature"]
    
    playlist_df = pd.DataFrame(columns = playlist_features_list)
    
    #step2
    
    playlist = sp.user_playlist_tracks(creator, playlist_id)["items"]
    for track in playlist:
        # Create empty dict
        playlist_features = {}
        # Get metadata
        playlist_features["artist"] = track["track"]["album"]["artists"][0]["name"]
        playlist_features["album"] = track["track"]["album"]["name"]
        playlist_features["track_name"] = track["track"]["name"]
        playlist_features["track_id"] = track["track"]["id"]
        
        # Get audio features
        audio_features = sp.audio_features(playlist_features["track_id"])[0]
        for feature in playlist_features_list[4:]:
            playlist_features[feature] = audio_features[feature]
        
        # Concat the dfs
        track_df = pd.DataFrame(playlist_features, index = [0])
        playlist_df = pd.concat([playlist_df, track_df], ignore_index = True)

    #Step 3
        
    return playlist_df

In [32]:
call_playlist("spotify", "37i9dQZEVXbMDoHDwVN2tF")

,artist,album,track_name,track_id,danceability,energy,key,loudness,mode,speechiness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature
0,Harry Styles,As It Was,As It Was,4LRPiXqCikLlN15c3yImP7,0.520,0.731,6,-5.338,0,0.0557,0.001010,0.3110,0.6620,173.930,167303,4
1,Jack Harlow,First Class,First Class,1rDQ4oMwGJI7B4tovsBOxc,0.905,0.563,8,-6.135,1,0.1020,0.000010,0.1130,0.3240,106.998,173948,4
2,Glass Animals,Dreamland (+ Bonus Levels),Heat Waves,02MWAaffLxlfxAUY7c5dvx,0.761,0.525,11,-6.900,1,0.0944,0.000007,0.0921,0.5310,80.870,238805,4
3,KAROL G,PROVENZA,PROVENZA,7dSZ6zGTQx66c2GF91xCrb,0.870,0.516,1,-8.006,1,0.0541,0.008230,0.1100,0.5300,111.005,210200,4
4,The Kid LAROI,F*CK LOVE 3: OVER YOU,STAY (with Justin Bieber),5PjdY0CKGZdEuoNab3yDmX,0.591,0.764,1,-5.484,1,0.0483,0.000000,0.1030,0.4780,169.928,141806,4
5,Camila Cabello,Familia,Bam Bam (feat. Ed Sheeran),2DB4DdfCFMw1iaR6JaR03a,0.756,0.697,8,-6.377,1,0.0401,0.000000,0.3330,0.9560,94.996,206071,4
6,Elton John,The Lockdown Sessions,Cold Heart - PNAU Remix,6JIC3hbC28JZKZ8AlAqX8h,0.795,0.800,1,-6.320,1,0.0309,0.000073,0.0915,0.9340,116.032,202735,4
7,Imagine Dragons,Mercury - Act 1,Enemy (with JID) - from the series Arcane Leag...,1HhNoOuqm1a5MXYEgAFl8o,0.728,0.783,11,-4.424,0,0.2660,0.000000,0.4340,0.5550,77.011,173381,4
8,Elley Duhé,MIDDLE OF THE NIGHT,MIDDLE OF THE NIGHT,58HvfVOeJY7lUuCqF0m3ly,0.410,0.611,4,-8.271,0,0.0467,0.000000,0.1160,0.0899,185.727,184448,3
9,Lizzo,About Damn Time,About Damn Time,1PckUlxKqWQs3RlWXVBLw3,0.836,0.743,10,-6.305,0,0.0656,0.000000,0.3350,0.7220,108.966,191822,4
